# Improvements

Perfusion<br>
Growth rate, and Product, the ratio figure<br>
Data export<br>

# Import Python Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# from CCDPApy.cell_line import bioprocess_pipeline
from CCDPApy import FedBatchParameters, FedBatchCellCulture, FedBatchCellLine, FedBatchExpriment

pd.set_option('display.max_columns', 200)

from CCDPApy.helper import input_path, split_df

In [41]:
path = input_path('fed_batch_data2.xlsx')
sheets = pd.read_excel(path, sheet_name=None)
sheets.keys()

dict_keys(['Measured Data', 'Feed Concentration', 'Polynomial Degree'])

In [2]:
param_1 = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False,
                             use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

param_2 = FedBatchParameters(cell_line_name='CL2',
                             use_concentration_after_feed=True,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

param_3 = FedBatchParameters(cell_line_name='CL3',
                             use_concentration_after_feed=False,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

path = input_path('fed_batch_data2.xlsx')
cl_handler = FedBatchCellCulture(parameters=[param_1, param_2,param_3])
cl_handler.load_data(file=path)

In [3]:
data_set = cl_handler.get_all_data()
data_set.keys()

dict_keys(['exp_data', 'feed_volume', 'conc_before_feed', 'conc_after_feed', 'measured_cumulative_conc', 'feed_conc', 'polynomial_degree_data'])

In [ ]:
cl_line_handler = FedBatchCellLine(cell_line_name='CL1', 
                                              data=data_set,
                                              use_feed_conc=True,
                                              use_conc_after_feed=False)
cl_line_handler.get_experiment_names()

In [ ]:
#cl_line_handler.in_process()
#cl_line_handler.polynomial()

In [ ]:
data_filtered = cl_line_handler.get_all_data()

In [ ]:
exp_handler = FedBatchExpriment(cell_line_name='CL1', 
                                        cell_line_id='13', 
                                        data=data_filtered,
                                        use_feed_conc=True,
                                        use_conc_after_feed=False)

In [ ]:
exp_handler.in_process()
exp_handler.polynomial()
exp_handler.rolling_window_polynomial(degree=3, window=6)

In [ ]:
exp_handler.get_conc_before_feed()

In [ ]:
s = exp_handler.get_species('glucose')
c = s.cumulative_conc['value'].values
t = s.run_time_hour

In [ ]:
s.conc

In [ ]:
s.growth_rate

In [ ]:
s.measurement_index

In [ ]:
s.conc_before_feed

In [ ]:
s.sp_rate

In [ ]:
s.volume_before_sampling

In [ ]:
from CCDPApy.helper import create_col_indices, split_name_unit

In [ ]:
exp_handler.get_conc_after_feed().shape

In [ ]:
a = exp_handler.get_measured_cumulative_conc()
create_col_indices(a)

In [ ]:
a.columns

In [ ]:
column_indices = {}
for i, col in enumerate(a.columns):
    name, unit = split_name_unit(col)
    print(i, name)
    column_indices[name] = {'index': i, 'unit': unit}

In [ ]:
a.shape

In [ ]:
a.head()

In [ ]:
a.iloc[:, 28]

In [ ]:
exp_handler.cumulative_cell_production

In [ ]:
exp_handler.polynomial()

In [ ]:
exp_handler.integral_viable_cell_conc

In [ ]:
exp_handler.growth_rate

In [ ]:
exp_handler.cumulative_conc

In [ ]:
spc = exp_handler.get_species('glutamine')
spc.sp_rate

In [ ]:
exp_handler.conc

In [ ]:
exp_handler.cumulative_conc

In [ ]:
exp_handler.sp_rate

In [ ]:
spc.polynomial(deg=3)

In [ ]:
data1 = spc.cumulative_conc_poly
data2 = spc.cumulative_conc
sns.lineplot(data=data1, x='Run Time (day)', y='value')
sns.scatterplot(data=data2, x='Run Time (day)', y='value')
plt.show()

In [ ]:
spc.sp_rate_poly['value']

In [ ]:
idx = spc.measurement_index
t = spc.run_time_hour[idx]
s = spc.cumulative_conc['value'].values[idx]
deg = 4
poly_func = np.poly1d(np.polyfit(x=t, y=s, deg=deg))
poly_func

In [ ]:
poly_deriv = poly_func.deriv()
y = poly_deriv(spc.run_time_hour)
xv = spc.viable_cell_conc['value'].values
v = spc.volume_before_sampling
y / (xv * v) * 1000

In [ ]:

0.044170892
0.033420916
0.024387564
0.018535344
0.012786592
0.007218002
0.005092897
0.002941554
0.001030772
-0.000441458
-0.00074635
0.000949806
0.004161749
0.012702173

In [ ]:
poly_func(spc.run_time_hour)

In [ ]:
spc.cumulative_conc_poly['value']

In [ ]:
spc.sp_rate_poly['value']

In [ ]:

0.044170892
0.033420916
0.024387564
0.018535344
0.012786592
0.007218002
0.005092897
0.002941554
0.001030772
-0.000441458
-0.00074635
0.000949806
0.004161749
0.012702173

In [ ]:
exp_handler.get_measured_data().head()

In [ ]:
cell = exp_handler.get_species(species='cell')
cell.in_process()

In [ ]:
cell.sp_rate

In [ ]:
igg = exp_handler.get_species(species='product')
igg.in_process()
igg.sp_rate

In [ ]:
glt = exp_handler.get_species(species='glucose')
glt.in_process(True, False)

In [ ]:
glt.sp_rate

# MeasuredData Class Test

## Cell Line 1

In [ ]:
from CCDPApy.MeasuredData.Fed_batch.MeasuredData import MeasuredData
from CCDPApy.helper.helper import input_path

# Read all sheets into a dictionary
file_path = input_path('CL1.xlsx')
sheets_dict = pd.read_excel(file_path, sheet_name=None)

for sheet_name, df in sheets_dict.items():
    md = MeasuredData(data=df)

In [ ]:
from CCDPApy.helper import create_col_indices
create_col_indices(md.separate_feed_df)

In [ ]:
md.separate_feed_df.iloc[:, 0]

In [ ]:


a = create_col_indices(md.conc_before_feed_df)
b = create_col_indices(md.conc_after_feed_df)
c = create_col_indices(md.feed_conc_df)
d = create_col_indices(md.measured_cumulative_conc_df)

In [ ]:
a.keys() == b.keys() and a.keys() == c.keys() and a.keys() == d.keys()


In [ ]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData

f1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
for file in f1:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data().head()

## Cell Line 2

In [ ]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData
f2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
for file in f2:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

## Cell Line 3

In [ ]:
f3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

for file in f3:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

# BioProcess Class Test

## In-Process

In in-process, the cumulative concentration will be calculated, <br>
and then specific rate will be calculated using the two-point calculation.

### Cell Line 1

There are the measurements of the feed concentration <br>
There are NOT measuremeents of the concentraion after feeding

In [ ]:
from CCDPApy.cell_line.Fed_batch import BioProcess

In [ ]:
from CCDPApy.cell_line.Fed_batch import BioProcess

files = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
#spc = ['Glucose','Lactate','Glutamine','Asparagine','Aspartate']

feed_concentration = True
concentration_after_feed = False

cell_line_1 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file,
                             #spc_list=spc,
                             #new_spc_list=['new'],
                             )
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_1[input_file.split('.')[0]] = bio_process
    
cell_line_1.keys()
exp_list = ['CL1_1', 'CL1_2', 'CL1_3']
exp = exp_list[0]

Display Experiment Information

In [ ]:
cell_line_1[exp].disp_data(exp_info=True)

Data about Cell

In [ ]:
cell = cell_line_1[exp].get_species('cell')
# DataFrame
cell.get_in_process.head()
# Display
# cell.disp_in_process

In [ ]:
cell.

Data about Oxygen

In [ ]:
oxy = cell_line_1[exp].get_species('oxygen')
# DataFrame
oxy.get_in_process
# Display
# oxy.disp_in_process

Data about Product(IgG)

In [ ]:
product = cell_line_1[exp].get_species('product')
# DataFrame
product.get_in_process
# Display
# product.disp_in_process

Data about other metabolite

In [ ]:
glc = cell_line_1[exp].get_species('glucose')
glc.get_in_process

In-process data

In [ ]:
bio_process.get_pre_process()

### Cell Line 2

In [ ]:
from CCDPApy import BioProcess

files = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']

feed_concentration = False
concentration_after_feed = True

cell_line_2 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_2[input_file.split('.')[0]] = bio_process

In [ ]:
cell = bio_process.get_species('cell')
cell.get_in_process

In [ ]:
igg = bio_process.get_species('product')
igg.get_in_process

In [ ]:
ala = bio_process.get_species('alanine')
ala.get_in_process

### Cell Line 3

In [ ]:
from CCDPApy import BioProcess

files = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

feed_concentration = False
concentration_after_feed = False

cell_line_3 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_3[input_file.split('.')[0]] = bio_process

## Display Pre-Process Data

In [ ]:
#bio_process.disp_data(exp_info=False, process='prepro')

## Post-Process

### Polynomial Regression

Cell Line 1

In [ ]:
cell_line_1

In [ ]:
poly_file = 'polynomial_order_1.xlsx'

for name, cl in cell_line_1.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
cell = cell_line_1['CL1_1'].get_species('cell')
cell.get_post_process

Cell Line 2

In [ ]:
cell_line_2

In [ ]:
poly_file = 'polynomial_order_2.xlsx'

for name, cl in cell_line_2.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
ala = cell_line_2['CL2_1'].get_species('alanine')
ala.get_in_process_data

Cell Line 3

In [ ]:
cell_line_3

In [ ]:
poly_file = 'polynomial_order_3.xlsx'

for name, cl in cell_line_3.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

### Rolling Window Polynomial Regression

Cell Line 1

In [ ]:
for name, cl in cell_line_1.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

In [ ]:
cl1_1 = cell_line_1['CL1_1']
cl1_1

In [ ]:
cl1_1.get_post_process_data()

In [ ]:
cell = cl1_1.get_species('cell')
cell.get_post_process_logistic

In [ ]:
bio_process.get_post_process_data()

In [ ]:
cl1_1 = cell_line_1['CL1_1']
ala = cl1_1.get_species('alanine')
ala._sp_rate_rollpoly, ala._t_mid

Cell Line 2

In [ ]:
for name, cl in cell_line_2.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

Cell Line 3

In [ ]:
for name, cl in cell_line_3.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

# Interactive Plot

In [ ]:
from CCDPApy import bioprocess_pipeline

key_cl1 = {'use_feed_conc': False,
           'use_conc_after_feed': False,}
cell_line = bioprocess_pipeline(cell_culture_type='fed-batch',
                                input_file_name='CL1.xlsx',
                                **key_cl1)
cell_line.interactive_plot()

In [ ]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline
from CCDPApy.BioProcess.BioProcess import BioProcess

input_files_1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
input_files_2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
input_files_3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']
measurement_sheet = 'Measured Data'
feed_sheet = 'Separate Feed Info'
poly_file = 'polynomial_order_1.xlsx'

cell_line = CellLine()

# Cell Line 1
for input in input_files_1:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=True, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 2
for input in input_files_2:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=True)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 3
for input in input_files_3:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)

cell_line.disp_cell_lines()

cell_line.interactive_plot(mode='inline')

## Growth rate

In [ ]:
import plotly.express as px
df = cell_line.get_plot_data()
cell_df = df[(df['species']=='cell')]
df1 = cell_df[(cell_df['profile']=='concentration') & (cell_df['kind']!='viability')]
df2 = cell_df[cell_df['kind']=='viability']

fig1 = px.line(df1, x='runTime', y='value', title='Cell concentration', markers=True,
              color='kind', line_dash='cellLine', symbol='runID')
fig1.update_layout(yaxis_title='(10^6 cells/mL)')

df2 = df2.rename(columns={'value': 'viability'})
fig2 = px.line(df2, x='runTime', y='viability', markers=True,
               color='cellLine', line_dash='cellLine', symbol='runID',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)

fig2.update_traces(yaxis='y2')
for fig_data in fig2.data:
    fig1.add_trace(fig_data)

fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'viability (%)', 'overlaying': "y",})
fig1

In [ ]:
df = cell_df[cell_df['profile']=='cumulative']
df1 = df[df['method']=='twoPoint']
df2 = df[df['method']=='polynomial']
fig1 = px.line(df1[df1['kind']=='integral'], x='runTime', y='value', 
                  title='Cell Cumulative concentration',
                  color='cellLine', line_dash='runID', symbol='cellLine')

fig2 = px.line(df1[df1['kind']=='cumulative'], x='runTime', y='value',
               color='cellLine', line_dash='runID', symbol='cellLine',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)
fig2.update_traces(yaxis='y2')

for fig_data in fig2.data:
    fig1.add_trace(fig_data)
fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'cumulative', 'overlaying': "y",})
fig1

In [ ]:
df = cell_df[(cell_df['profile']=='spRate') & (cell_df['kind']=='growthRate')]
fig = px.line(df, x='runTime', y='value', 
              title='Specific Growth Rate',
              color='cellLine', line_dash='runID', symbol='method')
fig